In [ ]:
%%capture
!pip install transformers

In [ ]:
%%capture
!pip install fastai2

In [ ]:
from dsnlplib import *

In [ ]:
import pandas
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import ReLU,add,BatchNormalization,concatenate,LSTM, Activation, Dense, Dropout, Input, Embedding, Conv1D, MaxPooling1D,GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import class_weight
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant

from keras.regularizers import l1, l2
#import ordinal_categorical_crossentropy as OC
from sklearn.utils import resample

Using TensorFlow backend.


In [ ]:
embeddings_index = dict()
f = open('data/itwac.csv')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1247492 word vectors.


In [ ]:
max_words = 100000		# da aumentare
max_len = 45			# dimensione massima testi era 20 / 45

In [ ]:
# Original data loading

cleaned=pandas.read_csv("data/DialogicScienceNLP-3classes.csv", header=0, index_col=False)#, dtype=f,parse_dates=[]
#sentences=pandas.read_csv("Dataset_2.0_unicode.csv", sep=";", header=0, index_col=False)#, dtype=f,parse_dates=[]

#cleaned.to_csv("DialogicScienceNLP-3classes.csv", index=False)


X=cleaned['Stralcio'].values
#numerical encoder
#le = LabelEncoder()
Yval=cleaned['Repertorio'].values

le = LabelEncoder()
Y = le.fit_transform(cleaned['Repertorio'].values)
onehot_encoder = OneHotEncoder(sparse=False)
#onehot_encoder = OrdinalEncoder()

integer_encoded = Y.reshape(len(Y), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded)
ordinal=onehot_encoded
#ordinal=np.zeros(onehot_encoded.shape)
#Yint=np.zeros((len(Yval), 1))
#for i in range(Yval.shape[0]):
#    if Yval[i]=="red":
#        ordinal[i,:]= np.array([1,0,0])
#        Yint[i:0]=0
#    if Yval[i]=="yellow":
#        ordinal[i,:]= np.array([0,1,0])
#        Yint[i:0]=1
#    if Yval[i]=="green":
#        ordinal[i,:]= np.array([0,0,1])
#        Yint[i:0]=2


X_train,X_test,Y_train,Y_test = train_test_split(X,ordinal,test_size=0.25, stratify=ordinal, random_state=42) #, shuffle=False)#

In [ ]:
n_labels=Y_train.shape[1]
tok = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'’»«”“…“„", lower=False)		# tokenizzazione + vectorizer
#tok = Tokenizer(num_words=max_words,lower=False, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n’»«”“…“„', split=" ")		# tokenizzazione + vectorizer

tok.fit_on_texts(X_train)					# considera i max_words termini più frequenti
sequences = tok.texts_to_sequences(X_train)	# i testi diventano sequenze di indici


In [ ]:


#print(sequences[0:10])

vocab_size = len(tok.word_index) + 1
print("Vocabulary",vocab_size)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print("Shape Sequence matrix",sequences_matrix.shape)


#loading with polyglot
#from polyglot.mapping import Embedding as Emb
#from polyglot.mapping import CaseExpander, DigitExpander

#embeddings = Emb.load(str("word2vec/polyglot-it.pkl"))
#embeddings = Emb.load(str("polyglot-it.pkl"))

#embeddings = embeddings.normalize_words()
#embeddings.apply_expansion(DigitExpander)
#embeddings.apply_expansion(CaseExpander)
#neighbors = embeddings.nearest_neighbors("è")
#print(neighbors)
#print(embeddings.distances("è", neighbors))
#exit(0)

# create a weight matrix for words in training docs
#embedding_matrix = np.zeros((max_words, 64)) # or 300
#notfoundwords=0
#for word, i in tok.word_index.items():
#    word=normalize_text(word)
#    if word in embeddings and i<max_words:
#        embedding_vector = embeddings[word]
#        if embedding_vector is not None:
#          #print("Word found:", word)
#            embedding_matrix[i] = embedding_vector
#    else:
#        print("Word not found:", word)
#        notfoundwords+=1
#print("Number of not found words", str(notfoundwords))
#print("Number of found words", str(len(tok.word_index.items())-notfoundwords))

# create a weight matrix for words in training docs
#embedding_matrix = np.zeros((max_words, 300)) # or 300
#notfoundwords=0
#for word, i in tok.word_index.items():
#    word=normalize_text(word)
#    if word in model.wv and i<max_words:
#        embedding_vector = model.wv[word]
#        if embedding_vector is not None:
#           #print("Word found:", word)
 #           embedding_matrix[i] = embedding_vector
#    else:
#        print("Word not found:", word)
#        notfoundwords+=1
#print("Number of not found words", str(notfoundwords))
#print("Number of found words", str(len(tok.word_index.items())-notfoundwords))



embedding_matrix = np.zeros((max_words, 128)) # or 300
notfoundwords=0
for word, i in tok.word_index.items():
    word=normalize_text(word)
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
                #print("Word found:", word)

                embedding_matrix[i] = embedding_vector
    else:
                print("Word not found:", word)
                notfoundwords+=1
print("Number of not found words", str(notfoundwords))
print("Number of found words", str(len(tok.word_index.items())-notfoundwords))

print("Embedding Matrix",embedding_matrix.shape)

#print(sequences[0])
#print(Y[0])

#definizione della rete

input=Input(shape=(max_len,))
e= Embedding(max_words, 128,  input_length=max_len,weights= [embedding_matrix], trainable=False)(input) #, input_length=max_len,

c1= Conv1D(150, 2, activation='relu', dilation_rate=1, padding="valid")(e) #5
bigram_branch = GlobalMaxPooling1D()(c1)
c_branch=c1

c1= Conv1D(300, 2, activation='relu', dilation_rate=1, padding="valid")(c_branch) #5
bigram_branch2 = GlobalMaxPooling1D()(c1)
c = concatenate([bigram_branch,bigram_branch2], axis=1)

# c1= Conv1D(50, 2, activation='relu', dilation_rate=1, padding="valid")(e) #5
# bigram_branch = GlobalMaxPooling1D()(c1)
# c2= Conv1D(50, 3, activation='relu', dilation_rate=1, padding="valid")(e) #5
# trigram_branch = GlobalMaxPooling1D()(c2)
# c3= Conv1D(50, 4, activation='relu', dilation_rate=1, padding="valid")(e) #5
# fourgram_branch = GlobalMaxPooling1D()(c3)
# c = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)
# c_branch = concatenate([c1, c2, c3], axis=1)
# c_branch = MaxPooling1D(3)(c_branch)
#
# c1= Conv1D(100, 2, activation='relu', dilation_rate=1, padding="valid")(c_branch) #5
# bigram_branch2 = GlobalMaxPooling1D()(c1)
# c2= Conv1D(100, 3, activation='relu', dilation_rate=1, padding="valid")(c_branch) #5
# trigram_branch2 = GlobalMaxPooling1D()(c2)
# c3= Conv1D(100, 4, activation='relu', dilation_rate=1, padding="valid")(c_branch) #5
# fourgram_branch2 = GlobalMaxPooling1D()(c3)
# c = concatenate([bigram_branch, trigram_branch, fourgram_branch,bigram_branch2, trigram_branch2, fourgram_branch2], axis=1)
# #c2=Conv1D(64,

#CNN model
#text_layer = Conv1D(64, 3, activation='relu')(e)
#text_layer = Conv1D(64, 3, activation='relu')(text_layer)
#text_layer = MaxPooling1D(3)(text_layer)
#text_layer = Conv1D(64, 3, activation='relu')(text_layer)
#text_layer = GlobalMaxPooling1D()(text_layer)
#text_layer = Dense(64, activation='relu')(text_layer)

out= Dense(n_labels, activation='softmax',kernel_regularizer=l2(0.0001))(c)#,kernel_regularizer=l2(0.001)
model = Model(inputs=input, outputs=out)


from keras import optimizers
myadam = optimizers.Nadam(lr=0.0001)# optimizers.SGD(lr=0.0001, momentum=0.9) #optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=myadam,
              metrics=['acc'])
model.summary()

# happy learning!
#model.fit(x_train, y_train, validation_data=(x_val, y_val),
#          epochs=2, batch_size=128)



#training
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_train.argmax(axis=1)),
                                                 Y_train.argmax(axis=1))

earlystop = EarlyStopping(monitor='val_loss',min_delta=0.0001, patience=10)
checkpoint = ModelCheckpoint('model', monitor='val_loss', save_best_only=True, save_weights_only=True)
reducelr= ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5 )

model.fit(sequences_matrix,Y_train,batch_size=32,epochs=200,
          validation_split=0.2,callbacks=[earlystop,checkpoint, reducelr], class_weight = class_weights) #, class_weight = class_weights
model.load_weights('model')
#test
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

y_score = model.predict(test_sequences_matrix)

matrix = confusion_matrix(Y_test.argmax(axis=1), y_score.argmax(axis=1))
#print("red","yellow","green")
print(le.classes_)
print(matrix)
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score, classification_report

print ("F1-score",f1_score(Y_test.argmax(axis=1), y_score.argmax(axis=1), average='macro'))
print ("Matthews CC",matthews_corrcoef(Y_test.argmax(axis=1), y_score.argmax(axis=1)))
print ("ROC AUC", roc_auc_score(Y_test, y_score, average='macro', multi_class='ovo'))
print(classification_report(Y_test.argmax(axis=1), y_score.argmax(axis=1), zero_division=0))

Vocabulary 12598
Shape Sequence matrix (3929, 45)
Word not found: 
Word not found: –
Word not found: farmi
Word not found: serd
Word not found: misselling
Word not found: Puschiasis
Word not found:  
Word not found: Ã¨
Word not found: Facebook
Word not found: di
Word not found: picchiarmi
Word not found: Assopopolari
Word not found: Vestager
Word not found: Cariferrara
Word not found: bullizzato
Word not found: difendermi
Word not found: Interverrei
Word not found: zonin
Word not found: che
Word not found: delle
Word not found: della
Word not found: coinvolgermi
Word not found: Legislativiche
Word not found: Raduzzi
Word not found: violenza
Word not found: Oettinger
Word not found: Katainen
Word not found: impanita
Word not found: Banca
Word not found: sfogarmi
Word not found:   il
Word not found: Frascerra
Word not found: adeguarmi
Word not found: rivolgendomi
Word not found: Confassociazioni
Word not found: fondo
Word not found: cabire
Word not found: aiutarmi
Word not found: Reagire